In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/Users/hirofumi48/162348.github.io/posts/2025/Comp"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| output: false
n, p, pₑ = 200, 50, 10

using Random, StatsFuns, Distributions
β_true = vcat(randn(pₑ), zeros(p - pₑ))
X = randn(n, p)

η_true = X * β_true
π_true = logistic.(η_true)

y = rand.(Bernoulli.(π_true))
y = collect(Float64, y)

200-element Vector{Float64}:
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 ⋮
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0
 0.0
 0.0
 0.0

In [3]:
#| output: false
using PolyaGammaHybridSamplers, LinearAlgebra, MCMCChains, Dates, MCMCDiagnosticTools

function pg_logistic_gibbs(
  X::Matrix{Float64},
  y::Vector{Float64};
  n_samples::Int = 5000,
  burnin::Int = 1000,
  σ_prior::Float64 = 10.0,
)
  n, p = size(X)

  # 事前: β ~ N(0, σ_prior^2 I)
  V0_inv = (1 / σ_prior^2) * LinearAlgebra.I  # precision of prior

  # 初期値
  β = zeros(p)
  κ = y .- 0.5  # κ_i = y_i - 1/2

  # サンプル保存用
  n_iter = n_samples + burnin
  β_samples = Matrix{Float64}(undef, n_samples, p)

  t_start = time()
  for it in 1:n_iter
    # 1. PG 補助変数 ω_i | β のサンプル
    η = X * β
    ω = similar(η)
    for i in 1:n
      pg = PolyaGammaHybridSampler(1.0, η[i])
      ω[i] = rand(pg)
    end

    # 2. β | ω, y のサンプル (多変量ガウス)
    Ω = Diagonal(ω)
    precision = X' * Ω * X + V0_inv          # posterior precision
    cov = inv(Matrix(precision))             # posterior covariance
    m = cov * (X' * κ)                       # posterior mean (μ0=0 のため)

    # β ~ N(m, cov)
    β = rand(MvNormal(m, Symmetric(cov)))

    # burn-in 後に保存
    if it > burnin
      β_samples[it - burnin, :] .= β
    end
  end
  t_stop = time()
  runtime_sec = t_stop - t_start

  names = Symbol.("β[$i]" for i in 1:p)
  values = reshape(β_samples, n_samples, p, 1)
  chain = Chains(values, names)
  chain = setinfo(chain, (
    start_time = [t_start],  # 1本チェインなら長さ1のベクトルでOK
    stop_time  = [t_stop],
  ))

  return chain, runtime_sec
end

pg_logistic_gibbs (generic function with 1 method)

In [4]:
σ_prior = 10.0
chain_pg, t_pg = pg_logistic_gibbs(X, y;
    n_samples = 3000,
    burnin = 3000,
    σ_prior = σ_prior,
)
summarize(chain_pg)



  parameters      mean       std      mcse   ess_bulk    ess_tail      rhat    ⋯
      Symbol   Float64   Float64   Float64    Float64     Float64   Float64    ⋯

        β[1]    1.1331    0.4784    0.0204   551.6891    869.6531    1.0017    ⋯
        β[2]    1.5660    0.4295    0.0240   320.9995    894.1765    1.0003    ⋯
        β[3]    1.2715    0.4343    0.0231   356.4819   1017.3785    1.0079    ⋯
        β[4]   -3.6429    0.6860    0.0508   183.4903    546.6948    1.0024    ⋯
        β[5]   -0.2386    0.3517    0.0147   571.2426   1065.7956    1.0014    ⋯
        β[6]   -3.4345    0.6611    0.0463   204.4201    391.6340    1.0055    ⋯
        β[7]   -1.0423    0.4653    0.0255   333.5981    801.7902    0.9999    ⋯
        β[8]   -4.3251    0.7773    0.0649   143.4168    325.3045    1.0048    ⋯
        β[9]    3.6844    0.6925    0.0523   178.9505    311.9926    1.0062    ⋯
       β[10]   -2.8290    0.5604    0.0402   193.8716    442.4915    1.0023    ⋯
       β[11]   -0.1154   

In [5]:
#| output: false
using Turing, LinearAlgebra

@model function logreg_turing(x, y, σ_prior)
    n, p = size(x)
    
    # 事前分布
    β ~ MvNormal(zeros(p), (σ_prior^2) * I)
    
    # ベクトル化した尤度（高速化）
    η = x * β
    y ~ arraydist(Bernoulli.(logistic.(η)))
end

model = logreg_turing(X, y, σ_prior)

DynamicPPL.Model{typeof(logreg_turing), (:x, :y, :σ_prior), (), (), Tuple{Matrix{Float64}, Vector{Float64}, Float64}, Tuple{}, DynamicPPL.DefaultContext}(Main.logreg_turing, (x = [0.4578346813796816 -0.4238453208320339 … -0.5676880524939117 0.6748713731069009; -0.4168672305736706 -0.3942168724654212 … -1.3758162614689953 1.9251817257264034; … ; 1.4935720821689815 0.029549273903954412 … -2.5558676915376517 -1.1775356880741032; 1.3983969506620484 1.8236323805189365 … -1.0109664989388292 0.16142346162794485], y = [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0], σ_prior = 10.0), NamedTuple(), DynamicPPL.DefaultContext())

In [6]:
#| output: false
n_samples = 3000
n_adapt   = 3000

chain_hmc = sample(
    model,
    NUTS(n_adapt, 0.6),
    n_samples,
)

┌ Info: Found initial step size
└   ϵ = 0.8


Sampling:   2%|▋                                        |  ETA: 0:00:17

Sampling:   2%|▉                                        |  ETA: 0:00:19

Sampling:   2%|█                                        |  ETA: 0:00:20

Sampling:   3%|█▎                                       |  ETA: 0:00:22

...

Sampling:  98%|████████████████████████████████████████▏|  ETA: 0:00:00

Sampling:  98%|████████████████████████████████████████▍|  ETA: 0:00:00

Sampling:  99%|████████████████████████████████████████▊|  ETA: 0:00:00

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:21


Chains MCMC chain (3000×64×1 Array{Float64, 3}):

Iterations        = 3001:1:6000
Number of chains  = 1
Samples per chain = 3000
Wall duration     = 22.51 seconds
Compute duration  = 22.51 seconds
parameters        = β[1], β[2], β[3], β[4], β[5], β[6], β[7], β[8], β[9], β[10], β[11], β[12], β[13], β[14], β[15], β[16], β[17], β[18], β[19], β[20], β[21], β[22], β[23], β[24], β[25], β[26], β[27], β[28], β[29], β[30], β[31], β[32], β[33], β[34], β[35], β[36], β[37], β[38], β[39], β[40], β[41], β[42], β[43], β[44], β[45], β[46], β[47], β[48], β[49], β[50]
internals         = n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size, lp, logprior, loglikelihood

Use `describe(chains)` for summary statistics and quantiles.


In [7]:
summarize(chain_hmc)



  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

        β[1]    1.1336    0.4901    0.0104   2279.7983   2033.5301    0.9998   ⋯
        β[2]    1.5556    0.4201    0.0115   1370.0031   1701.6643    1.0009   ⋯
        β[3]    1.2562    0.4411    0.0099   2000.6569   2064.6749    1.0003   ⋯
        β[4]   -3.6114    0.6779    0.0213   1029.6716   1716.9180    0.9998   ⋯
        β[5]   -0.2211    0.3540    0.0061   3352.6013   2184.0888    0.9997   ⋯
        β[6]   -3.4083    0.6687    0.0205   1076.3620   1569.4254    0.9997   ⋯
        β[7]   -1.0595    0.4524    0.0109   1739.4303   2092.1058    1.0000   ⋯
        β[8]   -4.2747    0.7901    0.0266    893.5055   1484.1267    1.0001   ⋯
        β[9]    3.6238    0.7056    0.0214   1095.0553   1988.0092    1.0001   ⋯
       β[10]   -2.7858    0.5691    0.0171   1119.8311   1838.6019    1.0000   ⋯
       β[11]   -0.1051   

In [8]:
using Statistics

# 真の β との誤差
mean_hmc = vec(mean(Array(chain_hmc), dims=1))  # ここは実際のパラメータ名に合わせて調整
mean_pg = vec(mean(Array(chain_pg), dims=1))

println("‖β̂_HMC - β_true‖₂ = ", norm(mean_hmc .- β_true))
println("‖β̂_PG  - β_true‖₂ = ", norm(mean_pg  .- β_true))

# ランタイムや ESS の比較も：
ess_hmc = ess_rhat(chain_hmc)
ess_pg  = ess_rhat(chain_pg)
println("ESS/s (HMC) = ", mean(ess_hmc[:,:ess_per_sec]))
println("ESS/s (PG) = ", mean(ess_pg[:,:ess_per_sec]))

‖β̂_HMC - β_true‖₂ = 7.598504396888691
‖β̂_PG  - β_true‖₂ = 7.702300637161629
ESS/s (HMC) = 110.53787823543833
ESS/s (PG) = 340.51247553566327
